# Guia para a aula experimental
Neste guia trataremos do uso de um sensor virtual, que utiliza fotografias para o cálculo de pessoas em uma localidade. Nesse guia vocês poderão testar diversos modelos de Deep Learning, indo desde uma Efficient Net até uma Mask-RCNN em diversas fotografias diferentes: dos restaurantes da Unicamp (em tempo real) e alguns exemplos encontrados na Web de Toronto, Boston e Nova York.No final você poderá enviar o resultado da inferência para a plataforma Konker, completando o ciclo completo de análise na nuvem e envio de dados processados para uma plataforma de IoT.Para seguir a parte final do roteiro, você precisará de uma conta na Konker, que pode ser criada em https://demo.konkerlabs.net/.

Para rodar esse Guia, você pode executar as células (exceto as que contêm listas de seleção) até chegar na célula contendo as usuário e senha de acesso ao dispostivo na plataforma Konker. Essa é a única célula na qual é necessária modificação, no caso, com a escrita das credenciais.

Esse roteiro utiliza material do trabalho publicado pelo Google, que pode ser encontrado em: https://github.com/tensorflow/hub/blob/master/examples/colab/tf2_object_detection.ipynb

Licensed under the Apache License, Version 2.0 

###Vamos garantir primeiro que a versão instalada do Tensorflow é igual ou superior a 2.6:

In [ ]:
%%capture
!pip install -U "tensorflow>=2.5"

###Carregando as bibliotecas que serão usadas

In [ ]:
import os
import pathlib

import matplotlib
import matplotlib.pyplot as plt

import io
import time
import requests
import json
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from six.moves.urllib.request import urlopen

import tensorflow as tf
import tensorflow_hub as hub

tf.get_logger().setLevel('ERROR')

###Definindo a função de leitura das imagens, os modelos e as imagens:

In [ ]:
def load_image_into_numpy_array(path):
  image = None
  if(path.startswith('http')):
    response = urlopen(path)
    image_data = response.read()
    image_data = BytesIO(image_data)
    image = Image.open(image_data)
  else:
    image_data = tf.io.gfile.GFile(path, 'rb').read()
    image = Image.open(BytesIO(image_data))

  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (1, im_height, im_width, 3)).astype(np.uint8)


ALL_MODELS = {
'EfficientDet D0 512x512' : 'https://tfhub.dev/tensorflow/efficientdet/d0/1',
'EfficientDet D1 640x640' : 'https://tfhub.dev/tensorflow/efficientdet/d1/1',
'EfficientDet D2 768x768' : 'https://tfhub.dev/tensorflow/efficientdet/d2/1',
'EfficientDet D3 896x896' : 'https://tfhub.dev/tensorflow/efficientdet/d3/1',
'SSD ResNet50 V1 FPN 640x640 (RetinaNet50)' : 'https://tfhub.dev/tensorflow/retinanet/resnet50_v1_fpn_640x640/1',
'SSD ResNet50 V1 FPN 1024x1024 (RetinaNet50)' : 'https://tfhub.dev/tensorflow/retinanet/resnet50_v1_fpn_1024x1024/1',
'SSD ResNet101 V1 FPN 640x640 (RetinaNet101)' : 'https://tfhub.dev/tensorflow/retinanet/resnet101_v1_fpn_640x640/1',
'SSD ResNet101 V1 FPN 1024x1024 (RetinaNet101)' : 'https://tfhub.dev/tensorflow/retinanet/resnet101_v1_fpn_1024x1024/1',
'Faster R-CNN ResNet50 V1 640x640' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet50_v1_640x640/1',
'Faster R-CNN ResNet50 V1 1024x1024' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet50_v1_1024x1024/1',
'Faster R-CNN ResNet50 V1 800x1333' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet50_v1_800x1333/1',
'Faster R-CNN ResNet101 V1 640x640' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet101_v1_640x640/1',
'Faster R-CNN ResNet101 V1 1024x1024' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet101_v1_1024x1024/1',
'Faster R-CNN ResNet101 V1 800x1333' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet101_v1_800x1333/1',
'Faster R-CNN Inception ResNet V2 640x640' : 'https://tfhub.dev/tensorflow/faster_rcnn/inception_resnet_v2_640x640/1',
'Faster R-CNN Inception ResNet V2 1024x1024' : 'https://tfhub.dev/tensorflow/faster_rcnn/inception_resnet_v2_1024x1024/1',
'Mask R-CNN Inception ResNet V2 1024x1024' : 'https://tfhub.dev/tensorflow/mask_rcnn/inception_resnet_v2_1024x1024/1'
}
COCO17_HUMAN_POSE_KEYPOINTS = [(0, 1), (0, 2), (1, 3), (2, 4), (0, 5), (0, 6), (5, 7), (7, 9), (6, 8), (8, 10), (5, 6), (5, 11), (6, 12), (11, 12), (11, 13), (13, 15), (12, 14), (14, 16)]
IMAGES_FOR_TEST = {
  'RU - Unicamp 1' : 'https://webservices.prefeitura.unicamp.br/cameras/cam_ru_a.jpg',
  'RU - Unicamp 2' : 'https://webservices.prefeitura.unicamp.br/cameras/cam_ru_b.jpg',
  'RA - Unicamp' : 'https://webservices.prefeitura.unicamp.br/cameras/cam_ra.jpg',
  'RS - Unicamp' : 'https://webservices.prefeitura.unicamp.br/cameras/cam_rs.jpg',
  'Toronto' : 'https://images.thestar.com/Sic8O1_yOZdZ_htIsDM7dfq33t0=/1280x1024/smart/filters:cb(1516743858004)/https://www.thestar.com/content/dam/thestar/life/food_wine/2012/10/04/toronto_restaurant_lineups_just_part_of_todays_eating_experience/grandelectric.jpeg',
  'dreamstime.com' : 'https://thumbs.dreamstime.com/z/nyc-people-waiting-outside-restaurant-table-20995159.jpg',
  'BostonGlobe' : 'https://bostonglobe-prod.cdn.arcpublishing.com/resizer/kOd-PMQUsCOkoEqK861NUX1XDQ4=/1440x0/arc-anglerfish-arc2-prod-bostonglobe.s3.amazonaws.com/public/2P7FQXFU3EI6HLDEZS3P4FVF7I.jpg',
}

###Clonando o repositório de modelos do tensorflow

In [ ]:
!git clone --depth 1 https://github.com/tensorflow/models

###Instalando a Image API do Tensorflow e compilando as bibliotecas Protobuf

In [ ]:
%%capture
%%bash
sudo apt install -y protobuf-compiler
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .


###Importando as bibliotecas de visualização

In [ ]:
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import ops as utils_ops

%matplotlib inline

###Importando as legendas do modelo

In [ ]:
PATH_TO_LABELS = './models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

###Na célula abaixo, você poderá selecionar o modelo de Deep Learning que será utilizado. Não é necessário rodar essa célula, basta apenas selecionar o modelo na lista. 
###Você pode voltar nessa célula em qualquer momento para testar o desempenho de outro modelo.

In [ ]:
#Agora vamos selecionar o modelo a ser usado
#@title Seleção do modelo { display-mode: "form", run: "auto" }
model_display_name = 'EfficientDet D0 512x512' # @param ['EfficientDet D0 512x512','EfficientDet D1 640x640','EfficientDet D2 768x768','EfficientDet D3 896x896','SSD ResNet50 V1 FPN 640x640 (RetinaNet50)','SSD ResNet50 V1 FPN 1024x1024 (RetinaNet50)','SSD ResNet101 V1 FPN 640x640 (RetinaNet101)','SSD ResNet101 V1 FPN 1024x1024 (RetinaNet101)','Faster R-CNN ResNet50 V1 640x640','Faster R-CNN ResNet50 V1 1024x1024','Faster R-CNN ResNet50 V1 800x1333','Faster R-CNN ResNet101 V1 640x640','Faster R-CNN ResNet101 V1 1024x1024','Faster R-CNN ResNet101 V1 800x1333','Faster R-CNN Inception ResNet V2 640x640','Faster R-CNN Inception ResNet V2 1024x1024','Mask R-CNN Inception ResNet V2 1024x1024']
model_handle = ALL_MODELS[model_display_name]

print('Modelo Selecionado:'+ model_display_name)
print('Link do modelo no TensorFlow Hub: {}'.format(model_handle))

In [ ]:
#Lendo o modelo selecionado
print('Lendo modelo...')
hub_model = hub.load(model_handle)
print('Modelo carregado!')

###Da mesma forma que na seleção do modelo, não é necessário rodar a célula abaixo, é necessário apenas selecionar a imagem na lista. As primeiras imagens da lista são obtidas em tempo real das câmeras dos restaurantes da Unicamp. Elas estão disponíveis publicamente na Web, nos endereços exibidos nesse notebook.
###As 3 últimas fotografias são imagens obtidas na Web das cidades de Toronto, Boston e Nova York. Elas também estão disponíveis na Web, nos endereços disponíveis nesse notebook.

In [ ]:
#@title Seleção da Imagem { display-mode: "form", run: "auto" }
selected_image = 'BostonGlobe' # @param ['RU - Unicamp 1', 'RU - Unicamp 2', 'RA - Unicamp','RS - Unicamp', 'Toronto', 'dreamstime.com', 'BostonGlobe']
flip_image_horizontally = False #@param {type:"boolean"}
convert_image_to_grayscale = False #@param {type:"boolean"}


###Obtendo a imagem da Web e exibindo no notebook

In [ ]:
image_path = IMAGES_FOR_TEST[selected_image]
print('Baixando: '+ selected_image + ' , em: '+ image_path)
image_np = load_image_into_numpy_array(image_path)


# Flip horizontally
if(flip_image_horizontally):
  image_np[0] = np.fliplr(image_np[0]).copy()

# Convert image to grayscale
if(convert_image_to_grayscale):
  image_np[0] = np.tile(
    np.mean(image_np[0], 2, keepdims=True), (1, 1, 3)).astype(np.uint8)

plt.figure(figsize=(12,16))
plt.imshow(image_np[0])
plt.show()

###Rodando a inferência do modelo escolhido

In [ ]:
results = hub_model(image_np)
result = {key:value.numpy() for key,value in results.items()}
#print(result.keys())

###Visualizando os resultados:
Obs: Você pode mudar o threshold de detecção alterando o valor da variável de mesmo nome abaixo

In [ ]:
label_id_offset = 0
image_np_with_detections = image_np.copy()

#Você pode mudar o threshold de detecção mudando o parâmetro abaixo:
threshold = 0.5

keypoints, keypoint_scores = None, None
if 'detection_keypoints' in result:
  keypoints = result['detection_keypoints'][0]
  keypoint_scores = result['detection_keypoint_scores'][0]

viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_detections[0],
      result['detection_boxes'][0],
      (result['detection_classes'][0] + label_id_offset).astype(int),
      result['detection_scores'][0],
      category_index,
      use_normalized_coordinates=True,
      max_boxes_to_draw=200,
      min_score_thresh=threshold,
      agnostic_mode=False,
      keypoints=keypoints,
      keypoint_scores=keypoint_scores,
      keypoint_edges=COCO17_HUMAN_POSE_KEYPOINTS)

plt.figure(figsize=(12,16))
plt.imshow(image_np_with_detections[0])
plt.show()

###Contando agora o número de detecções de pessoas na imagem

In [ ]:
person_count = 0
for i,j in enumerate(results['detection_scores'][0]):
  if j > threshold:
    if results['detection_classes'][0][i] == 1:
      person_count = person_count+1

print('Pessoas detectadas: ' + str(person_count))

###Vamos agora fazer a primeira modificação em uma célula.
###Crie um dispositivo na plataforma Konker e escreva as credenciais do **dispositivo** na célula abaixo

In [ ]:
sensor_user = ''
sensor_pass = ''

###Definindo agora as funções que usaremos para enviar os dados via REST para a plataforma. Nessa função o nome do canal foi configurado como **person_detector**, mas isso pode ser modificado na variável **canal**.

In [ ]:
current_milli_time = lambda: int(round(time.time() * 1000))

def post_data(user, passwd):
  global model_display_name, selected_image, person_count
  canal = 'person_detector'
  ts = current_milli_time()
  data = json.dumps({'_ts': ts, 'model': model_display_name, 'image' : selected_image, 'person_count':person_count})
  req = requests.post('http://data.demo.konkerlabs.net:80/pub/'+user+'/'+ canal, auth=(user, passwd), data=data)


###Por último, vamos enviar os dados para a plataforma

In [ ]:
post_data(sensor_user, sensor_pass)

###Caso tudo tenha funcionado corretamente, você deve ver uma mensagem chegando no seu dispositivo da plataforma contendo os dados do nome da imagem, do modelo de Deep Learning, timestamp e contagem de pessoas. Agora você pode retornar nas células anteriores e modificar o modelo e/ou modificar a fotografia onde ele é aplicado.